In [1]:
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertConfig, BertTokenizer, AutoTokenizer
import datasets
from datasets import Dataset
import pandas as pd
import tensorflow as tf
import numpy as np
import nltk
import nltk.translate.meteor_score as meteor
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import keras
#import wandb
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
from transformers import BigBirdPegasusForConditionalGeneration, TFTrainer, TFTrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, TrainingArguments, BartForConditionalGeneration, DataCollatorForSeq2Seq

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
torch.cuda
torch.cuda.is_available()

True

In [4]:
# cuda issues with this model
#path = "/home/ubuntu/Documents/OliverVinzelberg/led_large_16384_pubmed_CTS_untouched_blind_bestsofar_3072_1024_2bs_20e"

# this model works
path = "/home/ubuntu/Documents/OliverVinzelberg/led_large_16384_pubmed_CTS_untouched_3072_1024_2bs_20e"
model = AutoModelForSeq2SeqLM.from_pretrained(path, local_files_only=True)
model.cuda()

tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)

In [5]:
def generate_sum(data):
    to_pred = tokenizer(data, padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    #to_pred = tokenizer(data, padding="max_length", max_length=3072, return_tensors="pt", truncation=True)
    input_ids=to_pred["input_ids"].cuda()
    attention_mask=to_pred["attention_mask"].cuda()
    #global attention on special tokens
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1
    
    predicted_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
    
    return tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

In [6]:
with open('/home/ubuntu/Documents/OliverVinzelberg/pickle_files/CTS_data_processed_removed_backslashN.pkl', 'rb') as handle:
    df_cts = pickle.load(handle)
    
cts_dataset = Dataset.from_pandas(df_cts[["text", "summary"]])

In [7]:
print(cts_dataset['text'][0])

EU Clinical Trials Register Clinical trial results: A Phase 2, 2-Stage, 2-Cohort Study of Talazoparib (BMN 673) Administered to Germline BRCA Mutation Subjects with Locally Advanced and/or Metastatic Breast Cancer Summary EudraCT number 2013-003076-12 Trial protocol GB ES Global end of trial date 31 October 2018 Results information Result version number v2 (current) This version publication date 07 September 2019 First version publication date 17 September 2017 Version creation reason Trial information Trial identification Sponsor protocol code 673-201 Additional study identifiers ISRCTN number - ClinicalTrials.gov id (NCT number) NCT02034916 WHO universal trial number (UTN) - Other trial identifiers Alias Study Number: C3441008 Notes: Sponsors Sponsor organisation name Pfizer Inc. Sponsor organisation address 235 E 42nd Street, New York, United States, NY 10017 Public contact Pfizer ClinicalTrials.gov Call Center, Pfizer Inc., 001 8007181021, ClinicalTrials.gov_Inquiries@pfizer.com Sc

In [9]:
print(len(cts_dataset['text'][0]))

53440


In [10]:
print(len(cts_dataset['text'][0][0:4096]))

4096


In [11]:
print(len(cts_dataset['text'][0][4096:4096+4096]))

4096


In [32]:
no_chunks = 4096

test_list = cts_dataset['text'][0]

final_list= lambda test_list, no_chunks: [test_list[i:i+no_chunks] for i in range(0, len(test_list), no_chunks)]

output=final_list(test_list, no_chunks)
print(len(output[13]))
output[13]

192


' there any global interruptions to the trial? No Limitations and caveats None reported Clinical trial results 2013-003076-12 version 2 EU-CTR publication date: 07 September 2019 Page 31 of 31 '

In [8]:
print(cts_dataset['summary'][0])

 WHY WAS THIS STUDY DONE? Breast cancer is one of the most common causes of cancer in women. In 2012, around 1.7 million women were diagnosed with this disease around the world. Men can also get breast cancer, though this is quite rare.  Some breast cancers are caused by “mutations” in “DNA” in the BReast CAncer “genes” 1 and 2, which are more commonly known as BRCA1 and BRCA2. DNA is a material inside the cells of all living things that controls how cells in the body grow.  Genes are parts of DNA that contain the code for a specific feature, like eye color.  Mutations are changes in the DNA that can alter how a cell will behave.  If the BRCA1 and BRCA2 genes have mutations then this may cause breast cancer.  These mutations can be inherited from a parent with this mutation or through chance, which is like the roll of a dice. Special proteins known as “poly (ADP-ribose) polymerase” or “PARP” are found in all cells, and these proteins can help damaged cells fix or repair themselves so t

In [9]:
#max_input_length = 3072
max_input_length = 4096
max_output_length = 1024

def longformer_preprocess_func_cts(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length)
    
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [10]:
tokenized_cts_dataset = cts_dataset.map(longformer_preprocess_func_cts, batched=True)
tokenized_cts_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

test_size=0.15
processed_tokenized_cts_dataset = tokenized_cts_dataset.shuffle(seed=42).train_test_split(test_size=test_size)

processed_tokenized_cts_dataset['test']["text"][0][0:3072]

  0%|          | 0/1 [00:00<?, ?ba/s]

'EU Clinical Trials Register Clinical trial results: A Phase 3, Randomized, Double-Blind, Placebo-Controlled Study of the Efficacy and Safety of 2 Doses of Tofacitinib (CP-690,550) or Adalimumab in Subjects With Active Psoriatic Arthritis Summary EudraCT number 2011-003668-55 Trial protocol BE CZ ES SK HU DE PL BG Global end of trial date 18 December 2015 Results information Result version number v1 (current) This version publication date 22 December 2016 First version publication date 22 December 2016 Trial information Trial identification Sponsor protocol code A3921091 Additional study identifiers ISRCTN number - ClinicalTrials.gov id (NCT number) NCT01877668 WHO universal trial number (UTN) - Notes: Sponsors Sponsor organisation name Pfizer, Inc. Sponsor organisation address 235 E 42nd Street, New York, United States, NY 10017 Public contact Pfizer ClinicalTrials.gov Call Center, Pfizer, Inc., 001 800-718- 1021, ClinicalTrials.gov_Inquiries@pfizer.com Scientific contact Pfizer Clini

In [ ]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [11]:
processed_tokenized_cts_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 162
    })
    test: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 29
    })
})

In [12]:
generate_sum(processed_tokenized_cts_dataset['test']["text"][0])

['WHY WAS THIS STUDY DONE? Psoriatic arthritis (or “PsA”) is a disease that causes the joints to become swollen, tender, painful, or inflamed. PsA is caused by the body’s immune system, which mistakenly attacks the joints instead of the rest of the body.  Tofacitinib is a medicine that is being studied as a possible treatment for PsA.   : 5 0  0 2 T  : n O    This study was done to learn more about the use of tofacitiniband (also known as CP-690,550 or adalimumab) in people with active PsA, compared to placebo.  1 0   2 - b e The study included adult men and women who: \uf0b7 Had active psoriatic arthralgia (also called PsA) for at least 3 months \uf0a7 Were not responding to conventional synthetic disease-modifying \n anti-rheumatic drugs (ie, methotrexate, sulfasalazine, leflunomide, or others) and were unable to take \n  : 9 1  : 2 0  3 treated with an active control.  r p p A \\ d e v o r p A WHAT HAPPENED DURING THE STUDY?  d e 7 7 1 0 This study compared 3 groups of patients to f

In [13]:
processed_tokenized_cts_dataset['test']["summary"][0]

'WHY WAS THIS STUDY DONE? Psoriatic arthritis (also known as “PsA”) is a disease that can cause pain and swelling (inflammation) in or around the joints, tendon areas, fingers, toes, and back, and is often combined with itchy, scaly rashes on the skin. These problems are because the immune system, whose job is to attack foreign invaders like viruses and other germs, mistakenly attacks other parts of the body instead.  If PsA inflammation is left untreated, joints, tendon areas, fingers, and toes may become painful or swollen; skin rashes may occur; and people may become extremely tired. There is no cure for PsA at this time, but common treatments for PsA include medicines that control pain, reduce inflammation, and prevent the immune system from attacking the joints. Tofacitinib is a medicine that may reduce the activity of the immune system. It is an ) oral (taken by mouth) medication used to treat adults with PsA. In this study, T M researchers wanted to learn more about the use of t

In [13]:
generate_sum(processed_tokenized_cts_dataset['test']["text"][0][0:3072])

['A study to test the combination of tiotropium and olodaterol using the Respimat® inhaler in people with chronic obstructive \npulmonary disease (COPD) who have different abilities to inhale\nEudraCT number:2019-001719-21 \nThis version of the Boehringer Ingelheim Clinical Study Results Announcer (Bemp) Manual \nlooks at the results of this study.\nThis is a summary of results from 1clinical study. \nWe thank all study participants. You helped us toanswer important questions about the study \nmethods and results.\nWhat was this study about?\nThis study was about finding out whether a combination of 2 inhaled medicines \ntaken together helps people with COPD improve their lung function.\nParticipants with COPd had difficulty breathing \nin both directions.\nWho took part in this study?\nPeople with COPDONATED with chronic \nobstructive pulmonary disease could participate in this \nstudy if they had difficulty in breathing in both the right \nway and in the left way.\nparticipants could

In [14]:
generate_sum(cts_dataset['text'][1])

['WHY WAS THIS STUDY DONE? A seizure is a sudden increase in electrical activity in the brain. Seizures can cause many different symptoms, such as shaking, loss of consciousness, or loss of control of body movements. A seizure occurs when the brain loses control of its normal electrical activity and begins to move around too fast or too slowly.  Pregabalin is an experimental drug that is being studied as an add-on treatment for partial onset seizures (positions) in children with epilepsy. Pregaban is used to treat partial onset epilepsy in children and teenagers. The main purpose of this study was to learn more about the use of pregabalin in reducing the frequency of these seizures in children who have never had a seizure before.  WHAT HAPPENED DURING THE STUDY?  This study compared 2 groups of children to find out if a combination of Pregaba - nmentin (also known as no3AX16) and placebo (also called pradaxel®) would reduce the number of seizures in these children compared to placebo. 

In [14]:
generate_sum("Some randon string just to see what happens.")

['Some randonneurs have been known to have a tendency to run faster than others in a race.1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,17 But what is the most common reason for this?13,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,45,46,47,49,50,51,52,53,54,55,56,57,59,60,61,63,64,65,66,67,68,69,70,70-year-old female racemongrels are known for their ability to run fast and high speeds.29, but the most frequent reason they do so is because of their size and weight.30, but it is unknown if this is the case for male racemen.41,42,43-female racemen are known to run much faster than their male counterparts.44,44- female racemen do not run much farther than their counterparts.49,49-male racemen who run farther than they can run alone are not known to do so.50,52-female sprinter racemen can run farther and faster than any other racemen, but they can not run as fast as they can when they are in close proximity to each other.55,52: 53,54-female spimmers run farther in 

In [16]:
import pickle

with open('processed_tokenized_cts_dataset.pkl', 'wb') as handle:
    pickle.dump(processed_tokenized_cts_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [15]:
import pickle

with open('processed_tokenized_cts_dataset.pkl', 'rb') as handle:
    processed_tokenized_cts_dataset = pickle.load(handle)
